Reference:
- https://hackernoon.com/launch-a-gpu-backed-google-compute-engine-instance-and-setup-tensorflow-keras-and-jupyter-902369ed5272

# GCP

Create gpu-enabled instance on GCP
- provide ssh public key
- set "jupyter" network tag
- add firewall rule to enable port 8888 connection for "jupyter" network tag

# Local

Use gcloud command to ssh into instance

---
gcloud compute --project "keen-hangar-197817" ssh --zone "us-west1-b" "instance-1"

---

# Remote


## Install `Miniconda`

---
cd /tmp

curl -O https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh

bash Miniconda3-latest-Linux-x86_64.sh

source ~/.bashrc

----

## Installing `CUDA` Toolkit (`9.0`)

- Follow instructions on: 

https://developer.nvidia.com/cuda-90-download-archive?target_os=Linux&target_arch=x86_64&target_distro=Ubuntu&target_version=1604&target_type=debnetwork

---
curl -O http://developer.download.nvidia.com/compute/cuda/repos/ubuntu1604/x86_64/cuda-repo-ubuntu1604_9.0.176-1_amd64.deb

sudo dpkg -i cuda-repo-ubuntu1604_9.0.176-1_amd64.deb

sudo apt-key adv --fetch-keys http://developer.download.nvidia.com/compute/cuda/repos/ubuntu1604/x86_64/7fa2af80.pub

sudo apt-get update

sudo apt-get install cuda-9-0

---

## Add path

---
cat <<EOF >> ~/.bashrc <br/>
export CUDA_HOME=/usr/local/cuda-9.0 <br/>
export LD_LIBRARY_PATH=\${CUDA_HOME}/lib64 <br/>
export PATH=\${CUDA_HOME}/bin:\${PATH} EOF <br/>
source ~/.bashrc

---

## Verify driver

nvidia-smi
nvcc --version

## Installing `cuDNN`

**Local**

Go to website to download:
https://developer.nvidia.com/rdp/cudnn-download

---
cd ~/Downloads/

---

Upload file from local to remote

---
gcloud compute scp `local_file_path` instance-1:~/`remote_file_path`

---

**OR** download from elsewhere **Remote**

---
curl -O https://s3.amazonaws.com/open-source-william-falcon/cudnn-9.0-linux-x64-v7.1.tgz  

tar xvzf cudnn-9.0-linux-x64-v7.1.tgz

---

copy the cuDNN library to the CUDA toolkit directory

---
sudo cp -P cuda/include/cudnn.h $CUDA_HOME/include

sudo cp -P cuda/lib64/libcudnn* $CUDA_HOME/lib64

sudo chmod u+w $CUDA_HOME/include/cudnn.h

sudo chmod a+r $CUDA_HOME/lib64/libcudnn*

---

## Prepare virtual env


**Local**

In [3]:
%%writefile environment-gpu.yml
name: carnd-term1
channels:
    - https://conda.anaconda.org/menpo
    - conda-forge
dependencies:
    - python=3.6
    - numpy
    - matplotlib
    - jupyter
    - scikit-learn
    - seaborn
    - pandas
    - pip:
        - tensorflow-gpu
        - keras

Overwriting environment-gpu.yml


Make dir on remote

---
mkdir /home/shiwang/test-gpu

---

copy to remote

---
gcloud compute scp environment-gpu.yml instance-1:/home/shiwang/test-gpu

---

Create virtual environment

---
conda env create -f environment-gpu.yml

---


Activate

---
source activate carnd-term1
#source deactivate

---

## Config jupyter


---
jupyter notebook --generate-config

nano ~/.jupyter/jupyter_notebook_config.py

**CHANGE**:

c.NotebookApp.ip = '*'

#c.NotebookApp.ip = 'localhost'

---

Start Notebook

- Remote: jupyter notebook
- Local: http://35.233.241.85:8888/

Note: Cannot be KPMG network !!!

## Enable github


Remote

---
ssh-keygen -o -a 100 -t ed25519 -f ~/.ssh/id_test_gpu -C shiwang@instance-1

cat ~/.ssh/id_test_gpu.pub

https://github.com/settings/keys



---